# 1. predicting adsorption for each adsorption data point using Vt&BET only

In [11]:
import sys
sys.path.insert(0,'./src/')
from model_sets import models,para_grids
from preprocessing import pred_dataset
from model_opt import model_comparison

In [15]:
models

[('SVR', SVR(max_iter=10000)),
 ('DT', DecisionTreeRegressor(random_state=42)),
 ('ADBR', AdaBoostRegressor(random_state=42)),
 ('GBR', GradientBoostingRegressor(random_state=42)),
 ('BG', BaggingRegressor(n_jobs=-1, random_state=42)),
 ('ETR', ExtraTreesRegressor(n_jobs=-1, random_state=42)),
 ('RF', RandomForestRegressor(n_jobs=-1, random_state=42)),
 ('LGBM', LGBMRegressor(random_state=42))]

In [17]:
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
base_feature = ['V','S','L','BET',]
condition_feature = ['Temp(K)','Pressure']
pysical_prop = np.array(['Vt','Vmeso','Vmic'])
combin_1,combin_2,combin3 = [pysical_prop[0]],[pysical_prop[1]],[pysical_prop[2]]
combin_4,combin_5,combin_6 = pysical_prop[[0,1]],pysical_prop[[0,2]],pysical_prop[[1,2]]
combin_7 = pysical_prop

feature_list = [base_feature+combin_1+condition_feature,base_feature+condition_feature]
columns = ['Gas','Model_name','CV_r2','CV_mse','test_r2_total_model','test_mse_by_total_model','test_r2_separa_model','test_mse_separa_model','best_param']
#file_name = ['Total',"Meso","Micro",'All','Vmic_meso']
file_name = ['BET_only','BET_plut_Vt']
feature_set = ["BET","Vt",]
gas_list = ['CO2','CFCs','Methane','E&E']
results = []
for i in tqdm(range(10)):
    train_df,test_df = pred_dataset(['CO2-04-19-2022.xlsx','Methane-02-02-2022.xlsx','Ethane&Ethylene-02-02-2022.xlsx','CFCs-04-28-2022.xlsx'],feature_set,random_state = i)
    for j in range(len(feature_list)):
        result = model_comparison(train_df,test_df,[models[3],models[5],models[7]],para_grids, feature_list[j],gas_list)
        result.append(result)
files_name = 'GBR_ETR_LGBM_Full_Four_gases_with_pred_Vmic_'+file_name[j]+'_result_'+str(i)+'.csv'
pd.DataFrame(results,columns = columns).to_csv(os.path.join('./1_Predicting_separate_gas_by_two approach',files_name))  

  0%|          | 0/10 [00:00<?, ?it/s]

Dataset CO2, Algorithm GBR_total, Test_r2 0.9554, Test_error 0.4745


/Users/kai/miniforge3/envs/torch_env/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Dataset CO2, Algorithm ETR_total, Test_r2 0.941, Test_error 0.6246
Dataset CO2, Algorithm RF_total, Test_r2 0.9392, Test_error 0.6676
Dataset CFCs, Algorithm GBR_total, Test_r2 0.9422, Test_error 0.981
Dataset CFCs, Algorithm ETR_total, Test_r2 0.9167, Test_error 1.2766
Dataset CFCs, Algorithm RF_total, Test_r2 0.9005, Test_error 1.6977
Dataset Methane, Algorithm GBR_total, Test_r2 0.8891, Test_error 1.1028
Dataset Methane, Algorithm ETR_total, Test_r2 0.9024, Test_error 0.9201
Dataset Methane, Algorithm RF_total, Test_r2 0.8856, Test_error 1.0892
Dataset E&E, Algorithm GBR_total, Test_r2 0.9268, Test_error 0.5848
Dataset E&E, Algorithm ETR_total, Test_r2 0.8984, Test_error 0.7945
Dataset E&E, Algorithm RF_total, Test_r2 0.8725, Test_error 1.0012
Dataset CO2, Algorithm GBR_total, Test_r2 0.9439, Test_error 0.6166
Dataset CO2, Algorithm ETR_total, Test_r2 0.9221, Test_error 0.8249


/Users/kai/miniforge3/envs/torch_env/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Dataset CO2, Algorithm RF_total, Test_r2 0.9234, Test_error 0.8358
Dataset CFCs, Algorithm GBR_total, Test_r2 0.9462, Test_error 0.9145
Dataset CFCs, Algorithm ETR_total, Test_r2 0.9007, Test_error 1.4092
Dataset CFCs, Algorithm RF_total, Test_r2 0.9061, Test_error 1.5245
Dataset Methane, Algorithm GBR_total, Test_r2 0.909, Test_error 0.8502
Dataset Methane, Algorithm ETR_total, Test_r2 0.8941, Test_error 0.9612
Dataset Methane, Algorithm RF_total, Test_r2 0.8891, Test_error 1.0094
Dataset E&E, Algorithm GBR_total, Test_r2 0.9396, Test_error 0.4469
Dataset E&E, Algorithm ETR_total, Test_r2 0.8826, Test_error 0.9131


 10%|█         | 1/10 [04:22<39:23, 262.61s/it]

Dataset E&E, Algorithm RF_total, Test_r2 0.8762, Test_error 0.9462


 10%|█         | 1/10 [04:44<42:40, 284.51s/it]


KeyboardInterrupt: 

# Post result treatments
 

In [6]:
import pandas as pd
file_name = ['BET_only','BET_plut_Vt']
df_list = []
cal_columns= ["CV_r2","CV_mse","test_r2_separa_model","test_mse_separa_model"]
for j in range(1):
    for i in range(15):
    
        files_name = 'BG_ETR_Full_Four_gases_with_pred_Vmic_'+file_name[j]+'_result_'+str(i)+'.csv'
        df_list.append(pd.read_csv(os.path.join('./1_Predicting_separate_gas_by_two approach',files_name))[cal_columns] )
        pd.concat(df_list).groupby(level=0).mean().to_csv(os.path.join('./1_Predicting_separate_gas_by_two approach','mean.csv'))
        pd.concat(df_list).groupby(level=0).std().to_csv(os.path.join('./1_Predicting_separate_gas_by_two approach','std.csv'))


# 2_Predicting adsorption for each data point using the combination Vt, BET, Vmeso, and Vmic
the dataset for each separate gas will be smaller than previous one.

In [ ]:
import os
base_feature = ['V','S','L','BET',]
condition_feature = ['Temp(K)','Pressure']
combin_1 = ['Vt']
combin_2 = ["Vmeso"]
combin_3 = ['Vmic']
combin_4 = ['Vt',"Vmeso",]
combin_5 = ['Vt',"Vmic",]
combin_6 = ['Vt',"Vmic",'Vmeso',]
combin_7 = ["Vmic",'Vmeso',]

feature_list = [base_feature+condition_feature,base_feature+combin_1+condition_feature, \
    base_feature+combin_3+condition_feature, base_feature+combin_2+condition_feature,\
    base_feature+combin_4+condition_feature, base_feature+combin_5+condition_feature,\
    base_feature+combin_6+condition_feature, base_feature+combin_7+condition_feature, ]

columns = ['Gas','Model_name','CV_r2','CV_mse','test_r2_total_model','test_mse_by_total_model','test_r2_separa_model','test_mse_separa_model','best_param']


feature_set = ["BET","Vt","Vmeso","Vmic"]
gas_list = ['CO2','CFCs','Methane','E&E']
file_name = ['BET_only','BET_plut_Vt',"BET_Vmic","BET_Vmeso",'BET_Vt_Vmeso','BET_Vt_Vmic',"BET_Vt_Vmic_meso","BET_Vmic_meso"]


for i in range(10,15):
    train_df,test_df = pred_dataset(['CO2','Methane','Ethane&Ethylene','CFCs'],feature_set= feature_set)
    for j in range(len(feature_list)):
        results = model_comparison(models,para_grids, feature_list[j],gas_list)
        files_name = 'Four_gases_with_Vmic_'+file_name[j]+'_result_'+str(i)+'.csv'
        pd.DataFrame(results,columns = columns).to_csv(os.path.join('./2_Predicting_separate_gas_BET_Vt_Vmeso_Vmic',files_name))  
        #pd.DataFrame(results,columns = ['Gas','Algo','Train_erro','Test_error']).to_csv(os.path.join('./',files_name))   

# Post result treatment

In [24]:
import pandas as pd
file_name = ['BET_only','BET_plut_Vt',"BET_Vmic","BET_Vmeso",'BET_Vt_Vmeso','BET_Vt_Vmic',"BET_Vt_Vmic_meso","BET_Vmic_meso"]

cal_columns= ["CV_r2","CV_mse","test_r2_separa_model","test_mse_separa_model"]
for j in range(len(file_name)):
    df_list = []
    for i in range(11):
    
        files_name = 'Four_gases_with_Vmic_'+file_name[j]+'_result_'+str(i)+'.csv'
        df_list.append(pd.read_csv(os.path.join('./2_Predicting_separate_gas_BET_Vt_Vmeso_Vmic',files_name))[cal_columns] )
        pd.concat(df_list).groupby(level=0).mean().to_csv(os.path.join('./2_Predicting_separate_gas_BET_Vt_Vmeso_Vmic',file_name[j]+'_mean_new.csv'))
        pd.concat(df_list).groupby(level=0).std().to_csv(os.path.join('./2_Predicting_separate_gas_BET_Vt_Vmeso_Vmic',file_name[j]+'_std_new.csv'))


# Predicting the fitted parameters of adsorption isotherms

# Using only BET and Vt

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def pred_dataset(file_names, feature_set = feature_set):
    source_path = 'C:/Kai_Zhang/MachineLearning/Unified gas Adsorption/CO2_adsorption/new_data'
    train_df = pd.DataFrame()
    test_df = pd.DataFrame()
    for file_name in file_names:
        temp_data = pd.read_excel(os.path.join(source_path,file_name+'-fitting-02-02-2022.xlsx'),skiprows= 1 )
        temp_data = temp_data.dropna(axis=0,how = 'any',subset = feature_set)
        train_x,test_x = train_test_split(temp_data,test_size = 0.2)
        train_df = pd.concat([train_df,train_x],axis=0)
        test_df = pd.concat([test_df,test_x],axis =0)
    return train_df,test_df

In [ ]:
from sklearn.model_selection import GridSearchCV,cross_validate,GroupKFold
from sklearn.ensemble import ExtraTreesRegressor
from  sklearn.metrics import mean_squared_error,r2_score
from sklearn.utils import shuffle

def model_CV(train_x,train_y,model,para_grid):

    
    result = GridSearchCV(model,para_grid,cv= 5,
    scoring='neg_mean_squared_error', return_train_score=True,n_jobs=-1)
    result.fit(train_x,train_y)
    
    model_refit =model.set_params(**result.best_params_)
    train_cv = cross_validate(model_refit,train_x,train_y,cv =5,scoring = ('r2', 'neg_mean_squared_error'))
    train_mse_cv = -train_cv['test_neg_mean_squared_error'].mean()
    train_r2_cv = train_cv['test_r2'].mean()
    
    return [train_r2_cv,train_mse_cv],result.best_params_

# model evaluation
def model_eval(model,test_x,test_y):
      
    test_pre = model.predict(test_x)
    test_r2 = r2_score(test_pre,test_y)
    test_mse = mean_squared_error(test_y,test_pre)
    return test_r2,test_mse

# comparing different models
def model_comparison(model_list,para_grids,feature_list,gas_list):
    gas_list = gas_list 
    input_feature = feature_list
    output = ['Adsorp(mmol/g)']
    result_total = []

    for gas in gas_list:
        
        if gas =='total':

            train_df_com = train_df
            test_df_com = test_df
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output].values
            test_y = test_df_com[output].values
            
            train_x, train_y = shuffle(train_x, train_y,random_state=42)
            
            for model_name, model in model_list:

                
                result, best_param = model_CV(train_x,train_y.squeeze(),model,para_grids[model_name])
                model_refit = model.set_params(**best_param)
                model_refit.fit(train_x,train_y.squeeze())
                test_r2_total,test_mse_total = model_eval(model_refit,test_x,test_y.squeeze()) 
                for gases in gas_list[1:]:
                    test_df_com = test_df[test_df['Label']==gases]
                    test_xs = test_df_com[input_feature]
                    test_ys = test_df_com[output].values
                    test_r2,test_mse = model_eval(model_refit,test_xs,test_ys.squeeze()) 
                    result_total.append([gases,model_name+'_total',result[0],result[1],test_r2_total,test_mse_total,test_r2,test_mse,best_param])

                    print('Dataset {}, Algorithm {}, Test_r2 {}, Test_error {}'.format(gas,model_name+'_total',test_r2,test_mse))

            
        else:
            
            train_df_com = train_df[train_df['Label']==gas]
            test_df_com = test_df[test_df['Label']==gas]
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output].values
            test_y = test_df_com[output].values
            
            train_x, train_y = shuffle(train_x, train_y, random_state=42)
           
            for model_name, model in model_list:

                result, best_param = model_CV(train_x,train_y.squeeze(),model,para_grids[model_name])
                model_refit = model.set_params(**best_param)
                model_refit.fit(train_x,train_y.squeeze())
                test_r2,test_mse = model_eval(model_refit,test_x,test_y.squeeze()) 
                result_total.append([gas,model_name+'_separate',result[0],result[1],-1,-1, test_r2,test_mse,best_param])
                
                print(print('Dataset {}, Algorithm {}, Test_r2 {}, Test_error {}'.format(gas,model_name+'_total',test_r2,test_mse))
)
                
    return result_total